## Make and view auditory spectrograms

In [ ]:
import os
import numpy as np
import librosa
from audspec import Audspec
import matplotlib.pyplot as plt
from phonlab.utils import dir2df

## Customize

In [ ]:
wavdir = '/Users/ronald/src/audspec/resource/audio'  # Where to find the .wav files and textgrids.
specdir = '/Users/ronald/src/audspec/resource/spec'  # Where to stash the spectrograms
fs = 22050     # The .wav files will be resampled to the rate specified by `fs`.
step = 0.005  # The frame step for the spectrogram, in seconds.
maxpatterson_coef = 100
rfftparams = {            # Params passed to rfft() in make_zgram()
    'overwrite_x': True,  # Try to reduce memory usage
    'workers': -1         # Use all CPUs
}

## Create the objects used in the analysis

In [ ]:
# The Audspec to use for each of the .wav files.
aud = Audspec(fs, step_size=step, maxcbfiltn=maxpatterson_coef)

# Create image filters.
sharp_1 = aud.create_sharp_filter(span=3, mult=2)
sharp_2 = aud.create_sharp_filter(span=6, mult=1)
temporal_sharp = aud.create_sharp_filter(span=0.05, mult=1, dimension="time")
blur = aud.create_blur_filter(span=3, sigma=3)

## Find files to analyze

In [ ]:
wavdf = dir2df(wavdir, fnpat='\.wav$', addcols=['barename'])
print(f'Found {len(wavdf)} .wav files.')
wavdf

## Find cached spectrogram files

In [ ]:
specdf = dir2df(specdir, fnpat='(?P<wavname>.*)\.audspec\.npz$')
print(f'Found {len(specdf)} .npz files.')
specdf

## Merge to find .wav files that don't have a corresponding audspec file

In [ ]:
todo = wavdf.merge(specdf, how='left', left_on='barename', right_on='wavname', suffixes=['_wav', '_spec'])
todo = todo[todo['fname_spec'].isna()]
print(f'Found {len(todo)} .wav files that need audspec processing.')
todo

## Create and save spectrograms

In [ ]:
for row in todo.itertuples():
    data, _ = librosa.load(
        os.path.join(wavdir, row.relpath_wav, row.fname_wav),
        sr=fs
    )
#    break
    # Create auditory spectrogram.
    print('make_spect')
    aud.make_spect(data, rfftparams)
    print(f'made_spect of shape {aud.spect.shape}')
#    break
    print('make_zgram')
    zgram = aud.make_zgram()

    # Apply image filters to spectrogram.
    print('make_lgram')
    lgram = aud.apply_filt(zgram, sharp_1, axis=0, half_rectify=True)
    print('make_igram')
    igram = aud.apply_filt(lgram, blur, axis=0, half_rectify=False)
    print('make_lgram')
    lgram = aud.apply_filt(lgram, sharp_2, axis=0, half_rectify=True)

    print('make_ogram')
    ogram = aud.apply_filt(zgram, temporal_sharp, axis=1, half_rectify=True)

    # Save to disk. The `allow_pickle=False` param helps with portability.
    specname = os.path.join(specdir, f'{row.barename}.audspec.npz')
    np.savez(
        specname,
        spect=aud.spect,
        igram=igram,
        lgram=lgram,
        ogram=ogram,
        times=self.spect_times
    )
#    print('make_audspec')
#    np.save(specname, np.hstack((ogram, igram, lgram)), allow_pickle=False)
#    print(f'Saved audspec as {specname}')
#    specttimename = os.path.join(specdir, f'{row.barename}_audspec.times.npy')
#    print('make_audspec.times')
#    np.save(specttimename, aud.spect_times, allow_pickle=False)
#    print(f'Saved audspec.times as {specttimename}')